In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#referred to https://www.kaggle.com/satorushibata/optimize-lightgbm-hyperparameter-with-optuna-gpu to use gpu.

In [ ]:
#import libraries

import numpy as np
from collections import Counter
import pandas as pd
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score,accuracy_score,log_loss
from xgboost import XGBRegressor
import lightgbm as lgb
#optimizer 

from functools import partial
import optuna

In [ ]:
#load the training data 

train = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv",index_col =0)
test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv",index_col =0)

train.head()

In [ ]:
#for improving score according to the discussion (https://www.kaggle.com/c/tabular-playground-series-sep-2021/discussion/270206 ) 
train["nan_count"] =train.isnull().sum(axis=1)
test["nan_count"] =test.isnull().sum(axis=1)
train.head()

In [ ]:
#separate targets 
y = train['claim']
features = train.drop(['claim'],axis=1)

train.shape




In [ ]:
features_mean = features.mean()
features_std = features.std()
features_norm = (features - features_mean) /features_std

test_mean = test.mean()
test_std = test.std()
test_norm = (test - test_mean) /test_std
test_norm.head()

In [ ]:
#split the date to train and test
X_train,X_test,y_train,y_test = train_test_split(features_norm,y,test_size =0.3,random_state=0)


In [ ]:
# Get names of columns with missing values
#cols_with_missing = [col for col in X_train.columns
#                     if X_train[col].isnull().any()]
#len(cols_with_missing)

#118 columns include Nan values.

In [ ]:
#missing_val_count_by_column = (features.isnull().sum())
#print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
#an extension to imputation 

#X_train_plus = X_train.copy()
#X_test_plus = X_test.copy()

#cols_with_missing = [col for col in X_train.columns
#                     if X_train[col].isnull().any()]

#make new columns indicationg what will be imputed
#for col in cols_with_missing:
#    X_train_plus[col +'_was_missing'] =X_train_plus[col].isnull()
#    X_test_plus[col +'_was_missing'] = X_test_plus[col].isnull()

#from sklearn.impute import SimpleImputer

#my_imputer = SimpleImputer()
#imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
#imputed_X_test = pd.DataFrame(my_imputer.transform(X_test_plus))

#Imputation removed column name; put them back
#imputed_X_train.columns = X_train_plus.columns
#imputed_X_test.columns = X_test_plus.columns

In [ ]:
#missing_val_percents_by_column = features.isnull().sum()/len(features)
#print(missing_val_percents_by_column)

#for i in range(len(missing_val_percents_by_column)):
#    print(missing_val_percents_by_column[i])
# missing val percents of all the columns are closed to 1.6%




In [ ]:
# Reffered to https://www.kaggle.com/andreshg/xgboost-optuna-hyperparameter-tunning for hyperparameter tuning

In [ ]:
def objective(trial,X,y, name='xgb'):
    params = param = {
        'tree_method':'gpu_hist',  
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical(
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300, 3000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,15,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
    }

    model =  XGBRegressor(**params)
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=50,verbose=False)


    train_score = np.round(np.sqrt(mean_squared_error(y_train, model.predict(X_train))), 5)
    test_score = np.round(np.sqrt(mean_squared_error(y_test, model.predict(X_test))), 5)
                  
    print(f'TRAIN RMSE : {train_score} || TEST RMSE : {test_score}')
                  
    return test_score

In [ ]:
#%%time
#optimize = partial(objective,X=X_train,y=y_train)

#study_lgbm = optuna.create_study(direction ='minimize')
#study_lgbm.optimize(optimize,n_trials=50)

In [ ]:
#print(f"\tBest value (rmse): {study_lgbm.best_value:.5f}")
#print(f"\tBest params:")

#for key, value in study_lgbm.best_params.items():
#   print(f"\t\t{key}: {value}")
    

#Best value (rmse): 0.40686
#	Best params:
#		lambda: 1.7503469572793253
#		alpha: 0.044038476141827085
#		colsample_bytree: 0.8
#		subsample: 0.7
#		learning_rate: 0.016
#		n_estimators: 3000
#		max_depth: 5
#		min_child_weight: 104

In [ ]:
#setting parameters
params={
'lambda': 1.7503469572793253,
'alpha': 0.044038476141827085,
'colsample_bytree': 0.8,
'subsample': 0.7,
'learning_rate': 0.016,
'n_estimators': 3000,
'max_depth': 5,
'min_child_weight': 1043}

In [ ]:
model =  XGBRegressor(**params,tree_method='gpu_hist')
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

In [ ]:
#accuracy = accuracy_score(y_pred,y_test)
#accuracy1 = roc_auc_score(y_pred,y_test)
#print('LightGBM Model accuracy socre: {0:0.4f}'.format(accuracy))
#print('LightGBM Model accuracy socre: {0:0.4f}'.format(accuracy1))

#regression model score
mean_squared_error(y_test,y_pred,squared=False)


In [ ]:
#Making submittion.csv
                          
predictions = model.predict(test_norm)
output=pd.DataFrame({'Id' : test.index,'claim':predictions})
output.to_csv('submission.csv', index =False)